In [ ]:
%pip install -r requirements.txt
%pip install -q langchain-huggingface

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Path to your PDF folder
pdf_folder = "./data"

# Collect all documents from all PDFs
all_documents = []

for file in os.listdir(pdf_folder):
    if file.endswith(".pdf"):  # Only process PDFs
        loader = PyPDFLoader(os.path.join(pdf_folder, file))
        documents = loader.load()
        all_documents.extend(documents)

# Split the combined documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(all_documents)

print(f"Loaded {len(all_documents)} documents.")
print(f"Split into {len(texts)} chunks.")


In [ ]:
# Install the new package if not already installed
from langchain_huggingface import HuggingFaceEmbeddings

# Model you want to use
model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cuda'}  
encode_kwargs = {'normalize_embeddings': False}

# Initialize embeddings with the new class
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
import os
import time
from dotenv import load_dotenv
from langchain.vectorstores import Qdrant

# Load variables from .env
load_dotenv()

# Fetch values
url = os.getenv("QDRANT_URL")
api_key = os.getenv("QDRANT_API_KEY")

print(f"Number of text chunks to upload: {len(texts)}")
print(f"Using embedding model: {embeddings.model_name}")
print(f"Embedding device: {embeddings.model_kwargs['device']}")

# For large datasets, use smaller batches with progress tracking
batch_size = 50  # Can use larger batches with local HuggingFace model
total_batches = (len(texts) + batch_size - 1) // batch_size

print(f"Will process in {total_batches} batches of up to {batch_size} documents each")
print("Processing with local HuggingFace BGE model...")

try:
    # Create initial vector store with first batch
    first_batch = texts[:batch_size]
    print(f"Creating collection with batch 1/{total_batches} ({len(first_batch)} docs)")
    
    qdrant = Qdrant.from_documents(
        first_batch,
        embeddings,
        url=url,
        api_key=api_key,
        prefer_grpc=False,
        timeout=300,  
        collection_name="vector_db",
        force_recreate=True
    )
    
    uploaded_count = len(first_batch)
    print(f"✅ Batch 1 complete - {uploaded_count}/{len(texts)} documents uploaded")
    
    # Process remaining batches
    for i in range(1, total_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(texts))
        batch = texts[start_idx:end_idx]
        
        print(f"Processing batch {i+1}/{total_batches} ({len(batch)} docs)")
        
        # Extract text content and metadata
        batch_texts = [doc.page_content for doc in batch]
        batch_metadatas = [doc.metadata for doc in batch]
        
        # Add with retry logic
        max_retries = 3
        for attempt in range(max_retries):
            try:
                qdrant.add_texts(batch_texts, metadatas=batch_metadatas)
                uploaded_count += len(batch)
                print(f"✅ Batch {i+1} complete - {uploaded_count}/{len(texts)} documents uploaded")
                break
            except Exception as e:
                if attempt < max_retries - 1:
                    print(f"⚠️ Batch {i+1} failed (attempt {attempt+1}), retrying in 2 seconds...")
                    time.sleep(2)
                else:
                    print(f"❌ Batch {i+1} failed after {max_retries} attempts: {e}")
                    raise e
        
        # No delay needed for local embeddings
        
    print(f"\n🎉 Successfully uploaded all {uploaded_count} documents to Qdrant!")
    
except Exception as e:
    print(f"\n❌ Error during upload: {e}")
    if "timeout" in str(e).lower():
        print("This is a timeout error. Consider:")
        print("1. Using smaller batch sizes")
        print("2. Checking your internet connection")
        print("3. Upgrading your Qdrant instance if using cloud")
    elif "cuda" in str(e).lower() or "gpu" in str(e).lower():
        print("This might be a GPU/CUDA error. Try changing device to 'cpu' in the embeddings model.")
    else:
        print("Check your Qdrant connection and credentials")

In [ ]:
# Verify documents were uploaded to Qdrant
print(f"Using embedding model: {embeddings.model_name}")
print(f"Model device: {embeddings.model_kwargs['device']}")
print("="*50)

try:
    # Check collection info
    collection_info = qdrant.client.get_collection("vector_db")
    print(f"Collection Name: vector_db")
    print(f"Points Count: {collection_info.points_count}")
    print(f"Status: {collection_info.status}")
    
    # Check vector configuration
    if hasattr(collection_info.config.params, 'vectors'):
        if hasattr(collection_info.config.params.vectors, 'size'):
            print(f"Vector Size: {collection_info.config.params.vectors.size}")
        else:
            # Handle case where vectors is a dict
            vector_config = collection_info.config.params.vectors
            if isinstance(vector_config, dict):
                for key, config in vector_config.items():
                    print(f"Vector '{key}' Size: {config.size}")
            else:
                print(f"Vector Config: {vector_config}")
    
    print("\n" + "="*50)
    
    # Test similarity search with ocean-related queries
    print("Testing similarity search with BGE embeddings...")
    test_queries = [
        "ocean temperature measurements",
        "marine ecosystem monitoring", 
        "climate change impacts"
    ]
    
    for query in test_queries:
        print(f"\nQuery: '{query}'")
        results = qdrant.similarity_search(query, k=2)
        
        for i, result in enumerate(results, 1):
            print(f"  Result {i}:")
            print(f"    Content: {result.page_content[:150]}...")
            print(f"    Source: {result.metadata.get('source', 'Unknown')}")
    
    print(f"\n✅ Successfully verified {collection_info.points_count} documents in Qdrant!")
    print(f"✅ BGE embedding model ({embeddings.model_name}) is working correctly!")
    
except Exception as e:
    print(f"❌ Error verifying documents: {e}")
    print("The upload may have failed or the collection doesn't exist.")
    
    # Try alternative verification method
    try:
        print("\nTrying alternative verification...")
        # Just check if we can perform a search
        test_results = qdrant.similarity_search("test", k=1)
        if test_results:
            print(f"✅ Collection exists and search works! Found {len(test_results)} result(s)")
        else:
            print("⚠️ Collection exists but no documents found")
    except Exception as e2:
        print(f"❌ Alternative verification also failed: {e2}")
        if "cuda" in str(e2).lower():
            print("💡 Try changing the embedding device from 'cuda' to 'cpu'")